## knock 083 カテゴリごとに算出した中央値で欠損値を補完する

In [11]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [12]:
df_product = pl.read_csv("../docker/work/data/product.csv")
display(df_product.head())

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
str,i64,i64,i64,i64,i64
"""P040101001""",4,401,40101,198,149
"""P040101002""",4,401,40101,218,164
"""P040101003""",4,401,40101,230,173
"""P040101004""",4,401,40101,248,186
"""P040101005""",4,401,40101,268,201


### ノック

In [13]:
# 処理前の欠損値の確認
# unit_price と unit_cost に欠損値が含まれる
display(df_product.null_count())

df_filled = (
    df_product
    # カテゴリごとに算出した中央値で欠損値を補完する
    .with_columns([
        # coalesce: 第1引数の要素を順次代入して、欠損値があった場合は第2引数で補完する
        pl.coalesce([
            pl.col("unit_price"),
            # over: 統計値を計算するときの層別を指示する関数
            pl.median("unit_price").over("category_small_cd").round().cast(pl.Int64)
        ])
        .alias("unit_price"),
        pl.coalesce([
            pl.col("unit_cost"),
            pl.median("unit_cost").over("category_small_cd").round().cast(pl.Int64)
        ])
        .alias("unit_cost")    
    ])
)

# 処理後の欠損値の確認
display(df_filled.null_count())

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,7,7


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0
